## Introduction
In this article we will discuss a very common *decision problem*: how best to travel from a starting point `A` to some target destination `Z` in the *"best"* manner possible. Of course, as we previously discussed in the [last article](https://diogenesanalytics.com/blog/2025/02/26/reasoning-about-utility), it is the *utility* that must be defined in order to *quantitatively* define what is best. The majority of the article will present the *mathematical* definition of the utility function $U(x)$ for the *travel problem*, as well as the reasoning behind the definition, and finally a simple application.

## Utility Function Defined
Before we really get into explaining the *reasoning* behind our choice of *function definition*, we will first simply show the mathematical formula in all it's glory:

$$
U(x) = \frac{2D_{\text{ideal}} - D_{\text{actual}}}{M \cdot T \cdot S}
$$

Where:

- $D_{\text{ideal}}$ is the ideal (straight-line) distance between the start and end points of the journey.
- $D_{\text{actual}}$ is the actual traveled distance.
- $M$ is the monetary cost.
- $T$ is the total time.
- $S$ is the stress.

The *"non-simplified"* version looks like such:

$$
U(x) = \frac{D_{\text{ideal}}}{M \cdot T \cdot S} - \frac{D_{\text{actual}} - D_{\text{ideal}}}{M \cdot T \cdot S}
$$

For the next section, where we will be exploring the *"reasoning"* of each term in the equation, we will be exploring this less compact version.